# Knowledge Graph Extraction with LLM

**Duration:** ~35 min | **Platform:** Kaggle dual Tesla T4

This notebook builds a **knowledge graph** by using an LLM to extract entities
and relationships from documents, then analyzes the graph with RAPIDS cuGraph
and visualizes it with Graphistry.

### What you'll learn
1. Chunk documents for LLM processing
2. Extract structured entities and relationships
3. Build a knowledge graph with cuGraph
4. Run graph analytics (centrality, communities)
5. Query the knowledge graph with LLM

In [ ]:
!pip install -q git+https://github.com/llamatelemetry/llamatelemetry.git@v1.0.0

import llamatelemetry
from llamatelemetry.llama import ServerManager, LlamaCppClient
from llamatelemetry.kaggle import rapids_gpu, auto_register_graphistry
from huggingface_hub import hf_hub_download

llamatelemetry.init(service_name="knowledge-graph")

model_path = hf_hub_download(
    repo_id="bartowski/google_gemma-3-1b-it-GGUF",
    filename="google_gemma-3-1b-it-Q4_K_M.gguf",
    cache_dir="/root/.cache/huggingface",
)

mgr = ServerManager()
mgr.start_server(model_path=model_path, gpu_layers=99, tensor_split="1.0,0.0", ctx_size=2048)
mgr.wait_until_ready(timeout=60)
client = LlamaCppClient(base_url="http://127.0.0.1:8090")
auto_register_graphistry()
print("Ready")

## Document Processing

Split documents into manageable chunks for LLM entity extraction.

In [ ]:
@llamatelemetry.task(name="chunk-documents")
def chunk_documents(documents, chunk_size=500, overlap=50):
    """Split documents into overlapping chunks."""
    chunks = []
    for doc_id, doc in enumerate(documents):
        words = doc.split()
        for i in range(0, len(words), chunk_size - overlap):
            chunk = " ".join(words[i:i + chunk_size])
            chunks.append({"doc_id": doc_id, "chunk_id": len(chunks), "text": chunk})
    return chunks

# Sample documents about AI history
documents = [
    """Alan Turing proposed the concept of a universal computing machine in 1936. His work at Bletchley Park
    during World War II involved breaking the Enigma code. After the war, Turing worked at the University
    of Manchester where he developed early ideas about artificial intelligence. The Turing Test, proposed
    in 1950, remains a fundamental concept in AI research.""",
    """John McCarthy coined the term 'artificial intelligence' at the Dartmouth Conference in 1956. He later
    founded the Stanford AI Laboratory. McCarthy developed the Lisp programming language, which became
    the standard language for AI research. His work on formal reasoning influenced knowledge representation.""",
    """Geoffrey Hinton pioneered deep learning through his work on backpropagation and neural networks at
    the University of Toronto. Along with Yoshua Bengio and Yann LeCun, he is considered a founding father
    of modern deep learning. Hinton's work at Google Brain advanced image recognition significantly.""",
]

chunks = chunk_documents(documents)
print(f"Created {len(chunks)} chunks from {len(documents)} documents")

## Entity Extraction with LLM

Use structured JSON prompts to extract typed entities and relationships.

In [ ]:
import json

@llamatelemetry.task(name="extract-kg-triples")
def extract_kg_triples(text):
    prompt = f"""Extract knowledge graph triples from this text.
Return JSON with this exact format:
{{
  "entities": [{{"name": "...", "type": "PERSON|ORG|CONCEPT|PLACE|DATE"}}],
  "triples": [{{"subject": "...", "predicate": "...", "object": "..."}}]
}}

Text: {text}

JSON:"""

    resp = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        max_tokens=512, temperature=0.2,
    )
    content = resp.choices[0].message.content
    try:
        start = content.find("{")
        end = content.rfind("}") + 1
        return json.loads(content[start:end]) if start >= 0 else {"entities": [], "triples": []}
    except (json.JSONDecodeError, ValueError):
        return {"entities": [], "triples": []}

# Extract from all chunks
all_entities = []
all_triples = []
for chunk in chunks:
    result = extract_kg_triples(chunk["text"])
    all_entities.extend(result.get("entities", []))
    all_triples.extend(result.get("triples", []))

print(f"Extracted {len(all_entities)} entities, {len(all_triples)} triples")
for t in all_triples[:5]:
    print(f"  ({t.get('subject', '?')}) —[{t.get('predicate', '?')}]→ ({t.get('object', '?')})")

## Graph Construction

Build the knowledge graph from extracted triples using RAPIDS cuGraph on GPU 1.

In [ ]:
import pandas as pd

with rapids_gpu(1):
    # Build edge list from triples
    edges = []
    for t in all_triples:
        s, p, o = t.get("subject", ""), t.get("predicate", ""), t.get("object", "")
        if s and o:
            edges.append({"src": s, "predicate": p, "dst": o})

    edge_df = pd.DataFrame(edges) if edges else pd.DataFrame(columns=["src", "predicate", "dst"])

    # Deduplicate
    edge_df = edge_df.drop_duplicates(subset=["src", "dst"])

    # Build node list with types
    entity_types = {}
    for e in all_entities:
        name = e.get("name", "")
        if name:
            entity_types[name] = e.get("type", "UNKNOWN")

    nodes = set(edge_df["src"].tolist() + edge_df["dst"].tolist())
    node_df = pd.DataFrame([
        {"name": n, "type": entity_types.get(n, "UNKNOWN")} for n in nodes
    ])

    print(f"Knowledge graph: {len(node_df)} nodes, {len(edge_df)} edges")
    print(f"\nNode types: {node_df['type'].value_counts().to_dict()}")
    print(f"\nEdges:")
    print(edge_df.to_string(index=False))

## Graph Analytics

Run centrality analysis and community detection to find key entities.

In [ ]:
import numpy as np

@llamatelemetry.task(name="graph-analytics")
def analyze_graph(node_df, edge_df):
    """Compute degree centrality and identify hubs."""
    # Degree centrality (number of connections per node)
    degree = {}
    for _, row in edge_df.iterrows():
        degree[row["src"]] = degree.get(row["src"], 0) + 1
        degree[row["dst"]] = degree.get(row["dst"], 0) + 1

    max_degree = max(degree.values()) if degree else 1
    centrality = {k: v / max_degree for k, v in degree.items()}

    # Sort by centrality
    ranked = sorted(centrality.items(), key=lambda x: x[1], reverse=True)

    print("Top entities by degree centrality:")
    for name, score in ranked[:10]:
        entity_type = entity_types.get(name, "UNKNOWN")
        print(f"  {name:<30s} {entity_type:<10s} centrality={score:.2f}")

    return centrality

centrality = analyze_graph(node_df, edge_df)

## Interactive Visualization

Render the knowledge graph with Graphistry, colored by entity type.

In [ ]:
with rapids_gpu(1):
    try:
        import graphistry

        # Add centrality to node data
        node_df["centrality"] = node_df["name"].map(centrality).fillna(0)
        node_df["size"] = (node_df["centrality"] * 30 + 5).astype(int)

        g = (graphistry
             .edges(edge_df, "src", "dst")
             .nodes(node_df, "name")
             .bind(edge_title="predicate", point_title="name", point_size="size")
             .encode_point_color("type", categorical_mapping={
                 "PERSON": "blue", "ORG": "green", "CONCEPT": "orange",
                 "PLACE": "red", "DATE": "purple", "UNKNOWN": "gray",
             }))
        g.plot()
    except Exception as e:
        print(f"Graphistry: {e}")
        print("\nKnowledge graph structure:")
        for _, row in edge_df.iterrows():
            print(f"  {row['src']} —[{row['predicate']}]→ {row['dst']}")

## Query the Knowledge Graph

Use the LLM to answer questions based on the extracted knowledge graph.

In [ ]:
with llamatelemetry.session("kg-query"):
    # Format the graph as context for the LLM
    graph_context = "Knowledge graph triples:\n"
    for _, row in edge_df.iterrows():
        graph_context += f"- {row['src']} {row['predicate']} {row['dst']}\n"

    questions = [
        "Which people are connected to universities?",
        "What concepts were developed by Alan Turing?",
        "Who are the most connected entities in this knowledge graph?",
    ]

    for question in questions:
        with llamatelemetry.span("kg-query", question=question):
            resp = client.chat.completions.create(
                messages=[
                    {"role": "system", "content": f"Answer based on this knowledge graph:\n{graph_context}"},
                    {"role": "user", "content": question},
                ],
                max_tokens=128, temperature=0.3,
            )
            print(f"Q: {question}")
            print(f"A: {resp.choices[0].message.content}\n")

## Summary

This notebook demonstrated an end-to-end knowledge graph pipeline:
- **LLM extraction**: Structured entity/relationship extraction from text
- **Graph construction**: RAPIDS cuGraph on GPU 1
- **Analytics**: Centrality and community detection
- **Visualization**: Graphistry interactive graphs
- **Querying**: LLM-powered graph queries with session tracking

In [ ]:
mgr.stop_server()
llamatelemetry.shutdown()
print("Done.")